Import Data
- Make a dataframe from the csv
    - There is some inconsistent data (some rows formatted poorly), so we will skip them    


In [20]:
import pandas as pd

FILE = "./data/Assignment-1_Data.csv"

data = pd.read_csv(FILE, sep=";", on_bad_lines="skip", low_memory=False)

print(data)

        BillNo                             Itemname  Quantity  \
0       536365   WHITE HANGING HEART T-LIGHT HOLDER         6   
1       536365                  WHITE METAL LANTERN         6   
2       536365       CREAM CUPID HEARTS COAT HANGER         8   
3       536365  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4       536365       RED WOOLLY HOTTIE WHITE HEART.         6   
...        ...                                  ...       ...   
522059  581587          PACK OF 20 SPACEBOY NAPKINS        12   
522060  581587          CHILDREN'S APRON DOLLY GIRL         6   
522061  581587         CHILDRENS CUTLERY DOLLY GIRL         4   
522062  581587      CHILDRENS CUTLERY CIRCUS PARADE         4   
522063  581587         BAKING SET 9 PIECE RETROSPOT         3   

                    Date Price  CustomerID         Country  
0       01.12.2010 08:26  2,55     17850.0  United Kingdom  
1       01.12.2010 08:26  3,39     17850.0  United Kingdom  
2       01.12.2010 08:26  2,75     1

Clean Data
- Only data we care about is BillNo, Itemname, and Country 
    - We drop the other columns
- Remove entries that are missing data (BillNo, Itemname, or Country)
 

In [21]:
columns_to_keep = ['BillNo', 'Itemname', 'Country']

data = data[columns_to_keep]

#Drop rows with missing values
data.dropna(inplace=True)

print(data)

        BillNo                             Itemname         Country
0       536365   WHITE HANGING HEART T-LIGHT HOLDER  United Kingdom
1       536365                  WHITE METAL LANTERN  United Kingdom
2       536365       CREAM CUPID HEARTS COAT HANGER  United Kingdom
3       536365  KNITTED UNION FLAG HOT WATER BOTTLE  United Kingdom
4       536365       RED WOOLLY HOTTIE WHITE HEART.  United Kingdom
...        ...                                  ...             ...
522059  581587          PACK OF 20 SPACEBOY NAPKINS          France
522060  581587          CHILDREN'S APRON DOLLY GIRL          France
522061  581587         CHILDRENS CUTLERY DOLLY GIRL          France
522062  581587      CHILDRENS CUTLERY CIRCUS PARADE          France
522063  581587         BAKING SET 9 PIECE RETROSPOT          France

[520609 rows x 3 columns]


Group Data
- Group transaction data by country
- Find countries with a decent number of rows (transaction data)  
- We keep the countries with a lot of transaction data 
        - United Kingdom, France, and Germany  

In [22]:
country_datas = {country: data for country, data in data.groupby('Country')}

for country, data in country_datas.items():
        print(f"Number of Rows for {country}:")
        print(data.shape[0]) # number of rows
        print("\n")
        
countries_to_keep = ['United Kingdom', 'France', 'Germany']

country_datas = {key: value for key, value in country_datas.items() if key in countries_to_keep}

for country, df in country_datas.items():
        print(f"Data for {country}:")
        print(df) 
        print("\n")

Number of Rows for Australia:
1185


Number of Rows for Austria:
398


Number of Rows for Bahrain:
18


Number of Rows for Belgium:
2031


Number of Rows for Brazil:
32


Number of Rows for France:
8408


Number of Rows for Germany:
9042


Number of Rows for Greece:
145


Number of Rows for Hong Kong:
284


Number of Rows for Iceland:
182


Number of Rows for Israel:
295


Number of Rows for Italy:
758


Number of Rows for Japan:
321


Number of Rows for Lebanon:
45


Number of Rows for Lithuania:
35


Number of Rows for Malta:
112


Number of Rows for Netherlands:
2363


Number of Rows for Norway:
1072


Number of Rows for Poland:
330


Number of Rows for Portugal:
1501


Number of Rows for RSA:
58


Number of Rows for Saudi Arabia:
9


Number of Rows for Singapore:
222


Number of Rows for Spain:
2485


Number of Rows for Sweden:
451


Number of Rows for Switzerland:
1967


Number of Rows for USA:
179


Number of Rows for United Arab Emirates:
68


Number of Rows for United Kingdom:


Modify Data to be Transactions
- Make transactions by joining items with the same BillNo

In [23]:
country_transactions = {}

for country, data in country_datas.items():
    country_transactions[country] = data.groupby(['BillNo'])['Itemname'].apply(lambda x: ', '.join(x)).reset_index()
        
for country, transactions in country_transactions.items():
    transactions.drop(columns=['BillNo'], inplace=True)
    transactions.rename(columns={"Itemname": "Items"}, inplace=True)
    
for country, transactions in country_transactions.items():
    print(f"Transactions for {country}:")
    print(transactions) 
    print("\n")

Transactions for France:
                                                 Items
0    ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELIK...
1    PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI J...
2    EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PIN...
3    HOT WATER BOTTLE BABUSHKA, BREAD BIN DINER STY...
4    JAM MAKING SET PRINTED, SET/4 SKULL BADGES, RO...
..                                                 ...
387  LARGE RED BABUSHKA NOTEBOOK, SMALL RED BABUSHK...
388  SET OF 6 RIBBONS PERFECTLY PRETTY, MINI JIGSAW...
389  TRADITIONAL PICK UP STICKS GAME, GINGERBREAD M...
390  CAKE STAND WHITE TWO TIER LACE, REGENCY CAKEST...
391  CIRCUS PARADE LUNCH BOX, PLASTERS IN TIN CIRCU...

[392 rows x 1 columns]


Transactions for Germany:
                                                 Items
0    SET OF 6 T-LIGHTS SANTA, ROTATING SILVER ANGEL...
1    JAM MAKING SET PRINTED, JAM JAR WITH PINK LID,...
2    FELTCRAFT 6 FLOWER FRIENDS, 6 RIBBONS RUSTIC C...
3                     POSTAGE, JUMBO BAG RE